In [11]:
from pathlib import Path

def create_dir(dir, new):
    path = dir/ new
    path.mkdir(exist_ok=True)
    print(dir)
    return path

base_dir = create_dir(Path.cwd(), 'dataset')
train_dir = create_dir(base_dir, 'train')
test_dir = create_dir(base_dir, 'test')
valid_dir = create_dir(base_dir, 'validation')
georges_help_dir = create_dir(base_dir, 'georges_help')
non_georges_help_dir = create_dir(base_dir, 'non_georges_help')

train_georges_dir = create_dir(train_dir, 'georges')
train_non_georges_dir = create_dir(train_dir, 'non_georges')
test_georges_dir = create_dir(test_dir, 'georges')
test_non_georges_dir = create_dir(test_dir, 'non_georges')
valid_georges_dir = create_dir(valid_dir, 'georges')
valid_non_georges_dir = create_dir(valid_dir, 'non_georges')

/Users/olga/test_assignment_cv_engineer_data
/Users/olga/test_assignment_cv_engineer_data/dataset
/Users/olga/test_assignment_cv_engineer_data/dataset
/Users/olga/test_assignment_cv_engineer_data/dataset
/Users/olga/test_assignment_cv_engineer_data/dataset
/Users/olga/test_assignment_cv_engineer_data/dataset
/Users/olga/test_assignment_cv_engineer_data/dataset/train
/Users/olga/test_assignment_cv_engineer_data/dataset/train
/Users/olga/test_assignment_cv_engineer_data/dataset/test
/Users/olga/test_assignment_cv_engineer_data/dataset/test
/Users/olga/test_assignment_cv_engineer_data/dataset/validation
/Users/olga/test_assignment_cv_engineer_data/dataset/validation


Скачиваю файлы с ссылками картинки с гугл диск

In [ ]:
!wget "https://docs.google.com/uc?export=download&id=1RzK75yN2Fywrty3GoIfSajbfsgZULoQ5" -O georges.csv

In [ ]:
!wget "https://docs.google.com/uc?export=download&id=1nyfSC465u-W-DUeIKh8m5U44Z0SS2tyk" -O non_georges.csv

Импортирую скаачнные таблицы в txt файлы с соответствующими именами

In [2]:
import pandas as pd

df = pd.read_csv('georges.csv')
df.drop_duplicates(inplace=True)
df.to_csv("georges.txt", header=None, index=False)

In [13]:
import pandas as pd

df = pd.read_csv('non_georges.csv')
df.drop_duplicates(inplace=True)
df.to_csv("non_georges.txt", header=None, index=False)

In [ ]:
скачиваю картинки по ссылкам в txt файле в папку georges_help и последовательно переименовываю их в формат georges_1.jpg и 
non_georges_1.jpg, чтобы потом переложить в соответствующие папки

In [ ]:
!wget -P ./dataset/georges_help/ --random-wait -i georges.txt

In [ ]:
!wget -P ./dataset/non_georges_help/ --random-wait -i non_georges.txt

In [20]:
import pathlib
i = 0
for path in pathlib.Path("dataset/georges_help").iterdir():
    if path.is_file():
        #old_name = path.stem
#исходное имя файла
        old_extension = path.suffix
#исходное расширение файла
        directory = path.parent
#текущее расположение файла

        new_name = "georges" + str(i) + old_extension
        i += 1

        path.rename(pathlib.Path(directory, new_name))

In [15]:
import pathlib
i = 0
for path in pathlib.Path("dataset/non_georges_help").iterdir():
    if path.is_file():
        #old_name = path.stem
#исходное имя файла
        old_extension = path.suffix
#исходное расширение файла
        directory = path.parent
#текущее расположение файла

        new_name = "non_georges" + str(i) + old_extension
        i += 1

        path.rename(pathlib.Path(directory, new_name))

Перекладываю картинки из папки non_georges_help в папку georges_help

In [ ]:
from pathlib import Path
import shutil
def copy_files(src, dest, name, start, stop):
    fnames = [f'{name}{i}.jpg' for i in range(start,stop)]
    for fname in fnames:
        src_file = src / fname
        dest_file = dest / fname
        shutil.copy(src_file, dest_file)

src = Path.cwd() /'dataset'/ 'non_georges_help'

copy_files(src, georges_help, 'non_georges', 0, 3338)

In [ ]:
Перекладываю картинки georges и non_georges по папкам train, test, validation

In [31]:
from pathlib import Path
import shutil
def copy_files(src, dest, name, start, stop):
    fnames = [f'{name}{i}.jpg' for i in range(start,stop)]
    for fname in fnames:
        src_file = src / fname
        dest_file = dest / fname
        shutil.copy(src_file, dest_file)

src = Path.cwd() /'dataset'/ 'georges_help'
#train_georges_dir = Path.cwd() /'dataset'/'train'/'georges'

copy_files(src, train_georges_dir, 'georges', 0, 1100)
copy_files(src, train_non_georges_dir, 'non_georges', 0, 1670)
copy_files(src, test_georges_dir, 'georges', 1100, 1700)
copy_files(src, test_non_georges_dir, 'non_georges', 1670, 2500)
copy_files(src, valid_georges_dir, 'georges', 1700, 2300)
copy_files(src, valid_non_georges_dir, 'non_georges', 2500, 3330)

Извлекаю архитектуру VGG16

In [6]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet', # Источник весов
                  include_top=False, # Не подключать полносвязный слой
                  input_shape=(150, 150, 3)) # Форма входных тензоров

58892288/58889256 [==============================] - 1s 0us/step


Извлекаю признаки, выдаваемые сверточной основой архитектуры VGG16. Я указываю, что выходной признак должен иметь форму (образцы,4,4,512)

In [32]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
def extract_features(directory, sample_count, batch_size=20):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    for i, (inputs_batch, labels_batch) in enumerate(generator):
        if i * batch_size >= sample_count:
            # Генераторы могут находится в цикле до бесконечности,
            # поэтому нужно прервать после передачи всех изображений
            break
        # Предказываем на основе сверточной основы VGG16:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    return features, labels

In [ ]:
Для каждого набора изображений вызывается conv_base.predict для получения результата от сверточной основы. 
Этот результат я сохраняю в features и вызваю этот метод применительно к папкам с картинками:

In [34]:
train_features, train_labels = extract_features(train_dir, 2770)
valid_features, valid_labels = extract_features(valid_dir, 1430)
test_features, test_labels = extract_features(test_dir, 1430)

Found 2770 images belonging to 2 classes.
Found 1430 images belonging to 2 classes.
Found 1430 images belonging to 2 classes.


In [35]:
train_features = np.reshape(train_features, (2770, 4 * 4 * 512))
valid_features = np.reshape(valid_features, (1430, 4 * 4 * 512))
test_features = np.reshape(test_features, (1430, 4 * 4 * 512))

Полносвязный классификатор, который имеет на входе 256 нейронов и 1 на выходе. Ещё я добавила слой Dropout, чтобы избежать переобучения. 

In [36]:
from tensorflow.keras import (
    models, layers, optimizers)
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime
model = models.Sequential([
    layers.Dense(256, activation='relu', input_dim=4 * 4 * 512),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])
model.fit(train_features, train_labels,
          epochs=30,
          batch_size=20,
          validation_data=(valid_features, valid_labels),
          callbacks=[tensorboard_callback])

Epoch 1/30
139/139 [==============================] - 3s 17ms/step - loss: 0.6659 - acc: 0.6077 - val_loss: 0.5265 - val_acc: 0.7469
Epoch 2/30
139/139 [==============================] - 2s 12ms/step - loss: 0.5308 - acc: 0.7381 - val_loss: 0.4843 - val_acc: 0.7706
Epoch 3/30
139/139 [==============================] - 2s 12ms/step - loss: 0.4750 - acc: 0.7830 - val_loss: 0.4567 - val_acc: 0.7958
Epoch 4/30
139/139 [==============================] - 2s 12ms/step - loss: 0.4592 - acc: 0.7741 - val_loss: 0.4436 - val_acc: 0.8007
Epoch 5/30
139/139 [==============================] - 2s 12ms/step - loss: 0.3955 - acc: 0.8276 - val_loss: 0.4383 - val_acc: 0.8042
Epoch 6/30
139/139 [==============================] - 2s 13ms/step - loss: 0.4085 - acc: 0.8232 - val_loss: 0.4406 - val_acc: 0.8042
Epoch 7/30
139/139 [==============================] - 2s 13ms/step - loss: 0.4051 - acc: 0.8204 - val_loss: 0.4377 - val_acc: 0.8077
Epoch 8/30
139/139 [==============================] - 2s 13ms/step - 